# Strat discovery
Functionality:
- Review constants
- Run
- Allow saving progress and quitting
- End with visualizations and choosing which strats to go

## Imports

In [1]:
import discovery_downloader as ddl
import adf_calculator as adfc
import param_discovery as pmd
# from datetime import datetime
# import numpy as np
# import time

## Constants and Paramaters

In [2]:
## File names
PATHNAME = "../data/strat-discovery/"
HOURPATH = "../data/hour/"
MINUTEPATH = "../data/minute/"
ALL_ISOLATED_PAIRS = "all_isolated_pairs.csv"
ADF_STATS = "adf_stats.csv"
ADF_STATS_FILTERED = "adf_stats_filtered.csv"
DOWNLOAD_ORDER = "download_order.csv"
CURRENT_ADF = "current_adf.csv"
PAIR_RESULTS = "results.csv"

## Get all margin assets on Binance

In [12]:
ddl.download_all_available_margin_assets(PATHNAME, ALL_ISOLATED_PAIRS) #RUNTIME: 1 second

## Filter pairs by CoinMarketCap rank

In [3]:
ddl.filter_by_cmc_rank(PATHNAME, ALL_ISOLATED_PAIRS, cmc_rank=200)

Successfully filtered 88 assets out, only 143 remain


## Download hourly data

In [3]:
ddl.download_all_hourly_data(PATHNAME, ALL_ISOLATED_PAIRS, HOURPATH)

Operation took 27.74 minutes to download 231 files


## Compute ADF tests 

In [4]:
adfc.compute_adf_tests(PATHNAME, HOURPATH, ALL_ISOLATED_PAIRS, ADF_STATS, 10000)

Operation took 241.83 minutes


## Get filtered pairs

In [12]:
adfc.filter_adf_tests(PATHNAME, ADF_STATS, ADF_STATS_FILTERED, min_h=0, max_p=0.01, sort="p")

## Get download order

In [21]:
ddl.get_download_order(PATHNAME, ADF_STATS_FILTERED, DOWNLOAD_ORDER)

## Download minutely data in parallel
Go to Minute-Downloader and start downloading. Wait until the first two have finished downloadig before proceeding with below

http://localhost:8888/lab/tree/discovery/minute-downloader.ipynb

## Strat searching

In [3]:
RESTART = False
CONTINUE_FROM_LEFT_OFF = True

In [ ]:
pmd.brute_force_search(RESTART, CONTINUE_FROM_LEFT_OFF, PATHNAME, 
                       ADF_STATS_FILTERED, CURRENT_ADF, PAIR_RESULTS)

611 < 983


## Analysis

In [5]:
import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv(PATHNAME + ADF_STATS_FILTERED) #125, 245, 654

In [23]:
df

,A,B,t,p,h
0,CELRUSDT,FETUSDT,-5.538004,1.502226e-05,59.637962
1,ARDRUSDT,LSKUSDT,-4.958772,1.958992e-04,46.763503
2,ALGOUSDT,IOTXUSDT,-4.041288,6.265399e-03,445.433411
3,HOTUSDT,MTLUSDT,-11.449812,7.065478e-20,30.722250
4,NKNUSDT,WINUSDT,-10.916872,1.323903e-18,57.973449
...,...,...,...,...,...
745,MTLUSDT,XLMUSDT,-3.900852,9.892581e-03,289.031578
746,HBARUSDT,STMXUSDT,-3.900139,9.915006e-03,184.530375
747,ETHUSDT,FTMUSDT,-3.899228,9.943757e-03,421.988842
748,HOTUSDT,IOSTUSDT,-3.898429,9.969027e-03,304.892035


In [13]:
df1 = df.reindex([125, 245, 654])

In [14]:
df1

,A,B,t,p,h
125,CELRUSDT,FETUSDT,-5.538004,0.000015,59.637962
245,ARDRUSDT,LSKUSDT,-4.958772,0.000196,46.763503
654,ALGOUSDT,IOTXUSDT,-4.041288,0.006265,445.433411


In [15]:
df2 = df.drop([125, 245, 654])

In [19]:
df3 = df1.append(df2, ignore_index=True)

In [20]:
df3

,A,B,t,p,h
0,CELRUSDT,FETUSDT,-5.538004,1.502226e-05,59.637962
1,ARDRUSDT,LSKUSDT,-4.958772,1.958992e-04,46.763503
2,ALGOUSDT,IOTXUSDT,-4.041288,6.265399e-03,445.433411
3,HOTUSDT,MTLUSDT,-11.449812,7.065478e-20,30.722250
4,NKNUSDT,WINUSDT,-10.916872,1.323903e-18,57.973449
...,...,...,...,...,...
745,MTLUSDT,XLMUSDT,-3.900852,9.892581e-03,289.031578
746,HBARUSDT,STMXUSDT,-3.900139,9.915006e-03,184.530375
747,ETHUSDT,FTMUSDT,-3.899228,9.943757e-03,421.988842
748,HOTUSDT,IOSTUSDT,-3.898429,9.969027e-03,304.892035


In [21]:
df3.to_csv(PATHNAME + ADF_STATS_FILTERED, index=False)

In [26]:
df.reset_index(inplace=True, drop=True)

In [27]:
df

,A,B,t,p,h
0,MITHUSDT,OGNUSDT,-6.348362,2.774105e-07,50.108144
1,MTLUSDT,ONGUSDT,-6.342500,2.859457e-07,78.393491
2,DENTUSDT,MBLUSDT,-6.342336,2.861886e-07,59.123494
3,ANKRUSDT,MITHUSDT,-6.334893,2.974071e-07,104.863935
4,CTXCUSDT,TCTUSDT,-6.328173,3.079050e-07,48.885905
...,...,...,...,...,...
2474,CTSIUSDT,ZECUSDT,-3.337381,4.979716e-02,168.506946
2475,CHRUSDT,THETAUSDT,-3.337219,4.981764e-02,225.921638
2476,BNTUSDT,ONGUSDT,-3.336528,4.990561e-02,131.222281
2477,REPUSDT,TROYUSDT,-3.336256,4.994012e-02,107.192782


In [72]:
def get_download_order(pathname, adf_stat, filename):
    """saves the download order"""
    df = pd.read_csv(pathname + adf_stat)
    symbols = []
    for row in range(df.shape[0]):
        symbols.append(df.iloc[row].A)
        symbols.append(df.iloc[row].B)

    seen = set()
    seen_add = seen.add
    symbols = [x for x in symbols if not (x in seen or seen_add(x))]
    pd.Series(symbols).to_csv(pathname + filename, index=False)


In [74]:
len(symbols)

103

In [71]:
d

['DREPUSDT',
 'WINUSDT',
 'HOTUSDT',
 'MTLUSDT',
 'NKNUSDT',
 'ANKRUSDT',
 'WRXUSDT',
 'BTTUSDT',
 'OGNUSDT',
 'GTOUSDT',
 'CELRUSDT',
 'NPXSUSDT',
 'BTSUSDT',
 'CTXCUSDT',
 'CHZUSDT',
 'ENJUSDT',
 'MITHUSDT',
 'ETCUSDT',
 'RLCUSDT',
 'BNBUSDT',
 'VETUSDT',
 'ONEUSDT',
 'TFUELUSDT',
 'WANUSDT',
 'DOGEUSDT',
 'LSKUSDT',
 'TCTUSDT',
 'FETUSDT',
 'STPTUSDT',
 'BATUSDT',
 'WTCUSDT',
 'CTSIUSDT',
 'KEYUSDT',
 'AIONUSDT',
 'IOSTUSDT',
 'IOTAUSDT',
 'COCOSUSDT',
 'MFTUSDT',
 'KNCUSDT',
 'QTUMUSDT',
 'TRXUSDT',
 'THETAUSDT',
 'ARDRUSDT',
 'TROYUSDT',
 'ICXUSDT',
 'ZRXUSDT',
 'VITEUSDT',
 'CVCUSDT',
 'ARPAUSDT',
 'NANOUSDT',
 'XRPUSDT',
 'NULSUSDT',
 'HIVEUSDT',
 'MATICUSDT',
 'DENTUSDT',
 'HBARUSDT',
 'ONGUSDT',
 'MBLUSDT',
 'STMXUSDT',
 'BEAMUSDT',
 'ATOMUSDT',
 'FTMUSDT',
 'IOTXUSDT',
 'DUSKUSDT',
 'XLMUSDT',
 'EOSUSDT',
 'ETHUSDT',
 'PERLUSDT',
 'COSUSDT',
 'REPUSDT',
 'DOCKUSDT',
 'DASHUSDT',
 'NEOUSDT',
 'XMRUSDT',
 'LTOUSDT',
 'MDTUSDT',
 'XTZUSDT',
 'STXUSDT',
 'ADAUSDT',
 'WAVESUSDT',


In [31]:
150*10/60

25.0